In [1]:
%load_ext autoreload
%autoreload 2

import base64
import copy

import numpy as np
import plotly.graph_objects as go

from htc.models.image.DatasetImage import DatasetImage
from htc.settings_seg import settings_seg
from htc.tivita.DataPath import DataPath
from htc.tivita.hsi import tivita_wavelengths
from htc.utils.Config import Config
from htc.utils.visualization import add_std_fill, create_segmentation_overlay

In [2]:
paths = [DataPath.from_image_name("P044#2020_02_01_09_51_31")]
config = Config({
    "label_mapping": settings_seg.label_mapping,
    "input/n_channels": 100,
    "input/preprocessing": "L1",
})
img = DatasetImage(paths, train=False, config=config)[0]
hsi = img["features"].numpy()
labels = img["labels"].numpy()

label_mapping = copy.deepcopy(settings_seg.label_mapping)
label_mapping.label_colors = settings_seg.label_colors_paper
label_mapping.rename(settings_seg.labels_paper_renaming)

### Plot RGB overlaid with segmentation including opacity slider

In [3]:
fig_image = create_segmentation_overlay(labels, path=paths[0], label_mapping=label_mapping)
fig_image.update_layout(
    title=None,
    xaxis={"visible": False},
    yaxis={"visible": False},
    template="plotly_white",
    font_family="Libertinus Serif",
    font_size=16,
    margin=dict(l=10, r=10, b=10, t=20),
);

### Plot class-specific median and std spectra

In [4]:
fig_spectra = go.Figure()
for l in np.unique(labels):
    label = label_mapping.mapping_index_name[l]
    hsi_l = hsi[labels == l]
    median_l = np.median(hsi_l, axis=0)
    std_l = np.std(hsi_l, axis=0)
    add_std_fill(
        fig_spectra,
        x=tivita_wavelengths(),
        mid_line=median_l,
        std_range=std_l,
        linecolor=label_mapping.label_colors[label],
        label=label,
    )
fig_spectra.update_layout(
    template="plotly_white",
    font_family="Libertinus Serif",
    font_size=16,
    width=960,
    height=400,
    legend=dict(title=None, orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
    margin=dict(l=10, r=10, b=0, t=20),
)
fig_spectra.update_xaxes(title_text="<b>wavelength</b> [nm]")
fig_spectra.update_yaxes(title_text="<b>reflectance</b> [a.u.]");

### Create mapping to per-pixel spectrum

In [5]:
hsi_encoded = base64.b64encode(hsi.astype(np.float16))
hsi_decoded = base64.decodebytes(hsi_encoded)
hsi_decoded = np.frombuffer(hsi_decoded, dtype=np.float16)

np.allclose(hsi_decoded.reshape((480, 640, 100)), hsi)

True

### Create html

In [6]:
# js file containing the HSI cube. Stored as extra file to avoid browser freezing during loading
js_data = f"""const spectra_base64 = "{hsi_encoded.decode('ascii')}";""" + """

// There is no float16 array in JS, so we use uint16 and cast it later manually to float16
const spectra_data = new Uint16Array(Uint8Array.from(atob(spectra_base64), c => c.charCodeAt(0)).buffer);
"""

# JavaScript from the main document
js = """
// https://stackoverflow.com/a/8796597
function decodeFloat16(binary) {"use strict";
    var exponent = (binary & 0x7C00) >> 10,
        fraction = binary & 0x03FF;
    return (binary >> 15 ? -1 : 1) * (
        exponent ?
        (
            exponent === 0x1F ?
            fraction ? NaN : Infinity :
            Math.pow(2, exponent - 15) * (1 + fraction / 0x400)
        ) :
        6.103515625e-5 * (fraction / 0x400)
    );
};

document.addEventListener('DOMContentLoaded', function() {
    document.getElementById('loading').remove();

    img_div = document.getElementById('image');
    img_div.on('plotly_click', function(event) {
        let spectra_div = document.getElementById('spectra');
        const selected_point = event.points[0];
        const current_label = selected_point.text;

        // First hide all traces
        Plotly.restyle(spectra_div, {visible: 'legendonly'});

        // Show currently selected traces
        const traces = spectra_div.data;
        let traces_to_show = [];
        let traces_to_delete = [];
        for (let i = 0; i < traces.length; i++) {
            const trace = traces[i];

            if (trace.name == current_label) {
                traces_to_show.push(i);
            }

            if (trace.name == 'selected spectrum') {
                traces_to_delete.push(i);
            }
        }
        Plotly.restyle(spectra_div, {visible: true}, traces_to_show);

        // Show currently selected trace
        Plotly.deleteTraces(spectra_div, traces_to_delete);

        // Plotly uses the usual x-y-coordinate system, but the HSI data is stored with y down instead of up
        const y_transformed = (479 - selected_point.y);
        const start = (y_transformed * 640 + selected_point.x) * 100;
        const end = start + 100;
        let spectra = [];
        spectra_data.slice(start, end).forEach(function(x) {
            spectra.push(decodeFloat16(x));
        });
        const wavelengths = traces[0].x;

        Plotly.addTraces(spectra_div, [{
            x: wavelengths,
            y: spectra,
            name: 'selected spectrum',
            line: {color: 'black', dash: 'dash'}
        }]);
    });
});
"""

# The fonts also need to be copied to the resulting folder
css = """
/* Load custom font */
@font-face {
    font-family: libertinus;
    font-style: normal;
    src: url("fonts/LibertinusSerifDisplay-Regular.otf"), url("LibertinusSerifDisplay-Regular.otf");
}
@font-face {
    font-family: libertinus;
    font-weight: bold;
    src: url("fonts/LibertinusSerif-Semibold.otf"), url("LibertinusSerif-Semibold.otf");
}
body {
    font-family: libertinus, serif;
    hyphens: auto;
}
#loading {
    color: green;
}
#error {
    color: red;
}
figure {
    width: min-content;
    margin-bottom: 25px;
}
figure > figcaption {
    margin-left: 15px;
    margin-right: 15px;
}
figcaption {
    text-align: center;
    margin-top: 10px;
}
figcaption.main_caption {
    text-align: justify;
    margin-top: 25px;
}
"""

html = f"""
<!DOCTYPE html>
<html lang="en">
    <head>
        <title>Interactive spectra visualization</title>
        <meta charset="utf-8">
        <script defer src="interactive_example_spectra.js" onerror="document.getElementById('error').innerHTML = 'Could not load the image data. Please unzip all the suplementary material into the same folder and then open the HTML file with any browser.'"></script>
        <style>
        {css}
        </style>
    </head>
    <body>
        <p id="error"></p>
        <p id="loading">Loading. Please wait...</p>
        <figure>
            {fig_image.to_html(full_html=False, include_plotlyjs=True, div_id='image')}
            <figcaption><strong>(a)</strong> Example image with reference segmentation</figcaption>
        </figure>
        <figure>
            {fig_spectra.to_html(full_html=False, include_plotlyjs=False, div_id='spectra')}
            <figcaption><strong>(b)</strong> Spectrum exploration</figcaption>
            <figcaption class='main_caption'><strong>Supplementary Figure:</strong> Interactive exploration of a hyperspectral imaging (HSI) cube. (a) The RGB reconstruction of an exemplary HSI cube is overlaid with the reference segmentation map. The opacity of the overlay can be adjusted through the opacity slider. (b) By default, the mean spectrum and one standard deviation range is shown for each of the eight classes present on the example image. Mean spectra and their standard deviations are computed from all &#8467;1-normalized pixel spectra in the example image that belong to the respective class label. By clicking on the RGB image, the respective &#8467;1-normalized pixel spectrum (dashed line), the mean spectrum of the corresponding class (solid line) and a one standard deviation range (shaded area) are displayed. Median spectra and standard deviations from any class can be added/removed by clicking on the respective label name in the legend.</figcaption>
        </figure>
        <script>
            {js}
        </script>
    </body>
</html>"""

with (settings_seg.paper_dir / "interactive_example_spectra.html").open("w") as f:
    f.write(html)

with (settings_seg.paper_dir / "interactive_example_spectra.js").open("w") as f:
    f.write(js_data)